# Билет _

# Задание 2

In [21]:
import pandas as pd

In [22]:
sp500_df = pd.read_csv(
    'data/task_2/sp500hst.txt',
    sep=',',
    names=['date', 'ticker', 'open', 'high', 'low', 'close', 'volume']
)
sp500_df = sp500_df[sp500_df['ticker'] == 'NVDA']
sp500_df['date'] = pd.to_datetime(sp500_df['date'], format='%Y%m%d')
sp500_df = sp500_df.set_index('date')
sp500_df

,ticker,open,high,low,close,volume
date,,,,,,
2009-08-21,NVDA,13.87,13.95,13.5300,13.93,151858
2009-08-24,NVDA,13.87,14.00,13.4600,13.54,115369
2009-08-25,NVDA,13.58,13.80,13.3800,13.66,111026
2009-08-26,NVDA,13.66,13.85,13.5807,13.82,108580
2009-08-27,NVDA,13.88,14.07,13.7300,14.01,137787
...,...,...,...,...,...,...
2010-08-13,NVDA,9.25,9.52,9.2300,9.39,449437
2010-08-16,NVDA,9.28,9.45,9.0300,9.15,203558
2010-08-17,NVDA,9.23,9.44,9.1500,9.27,120368


In [23]:
date_open_min = sp500_df['open'].idxmin()
date_open_max = sp500_df['open'].idxmax()
start = min(date_open_min, date_open_max)
end = max(date_open_min, date_open_max)
start, end

(Timestamp('2009-12-31 00:00:00'), Timestamp('2010-08-12 00:00:00'))

In [24]:
period_df = sp500_df[(sp500_df.index >= start) & (sp500_df.index <= end)]
period_df.head()

,ticker,open,high,low,close,volume
date,,,,,,
2009-12-31,NVDA,18.79,18.95,18.66,18.68,175098
2010-01-04,NVDA,18.51,18.62,18.11,18.49,200088
2010-01-05,NVDA,18.50,18.96,18.42,18.76,182174
2010-01-06,NVDA,18.68,18.92,18.57,18.88,162291
2010-01-07,NVDA,18.78,18.86,18.37,18.51,136947


In [25]:
days = (end - start).days
volume = period_df['volume'].sum()
print(f'days={days}, volume={volume}')

days=224, volume=29025673


# Задание 3

In [114]:
import razdel
import dask.bag as db

In [115]:
def folder(counter, text):
    sents = razdel.sentenize(text)
    for sent in sents:
        end = sent.text[-3:]
        if end.endswith('.'):
            counter[2] += 1
        elif '?' in end:  # !?, ?! - считает за вопросительные из условия
            counter[0] += 1
        elif end.endswith('!'):
            counter[1] += 1

    return counter


def combiner(left, right):
    return left[0] + right[0], left[1] + right[1], left[2] + right[2]

In [118]:
%%time

bag = db.read_text(
    'data/task_3/all_k/*',
    encoding='cp1251',
    linedelimiter='987IV65~!`321'  # искусственно отменяем деление текста по строкам
)
bag = bag.fold(folder, combine=combiner, initial=[0, 0, 0])
res = bag.compute()
res

Wall time: 21.7 s


(368713, 188245, 3707067)

In [117]:
print(f'Вопросительные: {res[0]}, Побудительные: {res[1]}, Повествовательные: {res[2]}')

Вопросительные: 368713, Побудительные: 188245, Повествовательные: 3707067


In [99]:
# альтернативное решение через delayed медленнее:

In [109]:
import dask
import os
from functools import reduce

In [110]:
@dask.delayed(pure=True, traverse=False)
def count(path):
    with open(os.path.join('data/task_3/all_k/', path), encoding='cp1251') as f:
        text = f.read()

    counter = [0, 0, 0]
    for sent in razdel.sentenize(text):
        end = sent.text[-3:]
        if end.endswith('.'):
            counter[2] += 1
        elif '?' in end:  # !?, ?! - считает за вопросительные из условия
            counter[0] += 1
        elif end.endswith('!'):
            counter[1] += 1

    return counter

In [111]:
%%time

delayed = []
for path in os.listdir('data/task_3/all_k/'):
    delayed.append(count(path))

counters = dask.compute(delayed)
res = reduce(combiner, counters[0], [0, 0, 0])

Wall time: 1min 14s
